# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298350199624                   -0.85    5.1         
  2   -8.300187076041       -2.74       -1.25    1.1   84.2ms
  3   -8.300429691699       -3.62       -1.89    2.9    258ms
  4   -8.300462345550       -4.49       -2.78    2.1   87.4ms
  5   -8.300464496814       -5.67       -3.14    3.1    120ms
  6   -8.300464583139       -7.06       -3.32    9.0    161ms
  7   -8.300464612786       -7.53       -3.48    2.6    110ms
  8   -8.300464629024       -7.79       -3.63    1.2   96.6ms
  9   -8.300464638854       -8.01       -3.80    1.4    119ms
 10   -8.300464641671       -8.55       -3.93    1.1   75.8ms
 11   -8.300464643739       -8.68       -4.26    4.6    131ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64202297372                   -0.70    6.1         
  2   -16.67840060232       -1.44       -1.14    1.4    210ms
  3   -16.67921088053       -3.09       -1.88    3.2    241ms
  4   -16.67927726449       -4.18       -2.74    2.4    217ms
  5   -16.67928570627       -5.07       -3.09    5.0    329ms
  6   -16.67928618901       -6.32       -3.52    2.5    221ms
  7   -16.67928621611       -7.57       -4.01    2.1    220ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31850680880                   -0.56    6.6         
  2   -33.32577588046       -2.14       -1.00    1.2    715ms
  3   -33.33600220930       -1.99       -1.68    6.1    1.02s
  4   -33.33617193700       -3.77       -2.58    1.9    722ms
  5   -33.33690991525       -3.13       -2.57   10.5    1.34s
  6   -33.33693079754       -4.68       -2.78    3.1    758ms
  7   -33.33694353778       -4.89       -3.52    2.0    786ms
  8   -33.33694377055       -6.63       -3.84    3.4    863ms
  9   -33.33694378201       -7.94       -4.36    1.8    647ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298480015413                   -0.85    5.2         
  2   -8.300271741910       -2.75       -1.59    1.0   68.0ms
  3   -8.300444170239       -3.76       -2.72    3.1   97.4ms
  4   -8.300457757750       -4.87       -2.82    4.1    127ms
  5   -8.300464046474       -5.20       -3.31    1.0   71.2ms
  6   -8.300464592019       -6.26       -3.81    2.9    122ms
  7   -8.300464642881       -7.29       -4.56    2.6    114ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32698498802                   -0.56    7.5         
  2   -33.33093225005       -2.40       -1.28    1.2    595ms
  3   -27.64451553456   +    0.75       -0.69    6.6    1.12s
  4   -33.02370356736        0.73       -1.27    4.5    1.07s
  5   -33.29432960258       -0.57       -1.44    2.5    781ms
  6   -33.23532414672   +   -1.23       -1.55    2.5    753ms
  7   -33.17070840203   +   -1.19       -1.45    3.4    823ms
  8   -33.33348531264       -0.79       -2.23    3.1    759ms
  9   -33.33435790233       -3.06       -2.30    3.8    875ms
 10   -33.33469421050       -3.47       -2.34    1.4    595ms
 11   -33.33582287001       -2.95       -2.48    1.1    573ms
 12   -33.33682403856       -3.00      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.